In [2]:
import pytest

pytestmark = [pytest.mark.unit, pytest.mark.release]

import numpy as np
from thirdai import bolt

from utils import (
    gen_single_sparse_layer_network,
    gen_numpy_training_data,
    get_categorical_acc,
    train_network,
    build_simple_distributed_bolt_network,
    train_single_node_distributed_network,
)


network = build_simple_distributed_bolt_network(sparsity=1.0, n_classes=10)
examples, labels = gen_numpy_training_data(n_classes=10, n_samples=1000)
train_single_node_distributed_network(
    network, examples, labels, epochs=1, update_parameters=False
)

indices, values = network.get_indexed_sketch_for_gradients(
    layer_index=0, compression_density=0.1, sketch_biases=False, seed_for_hashing=0
)
network.set_gradients_from_indices_values(
    layer_index=0, indices=indices, values=values, set_biases=False
)

network_weights = network.get_weights_gradients(0).flatten()
np.add.at(network_weights, indices, -1 * values)
norm_after_subtracting_gradients = np.linalg.norm(network_weights)

assert norm_after_subtracting_gradients == 0


Initializing Bolt network...
========= Bolt Network =========
InputLayer (Layer 0): dim=10
FullyConnectedLayer (Layer 1): dim=50, sparsity=1, act_func=ReLU
FullyConnectedLayer (Layer 2): dim=10, sparsity=1, act_func=Softmax
Initialized Network in 0 seconds
Distributed Network initialization done on this Node
